
<a href="https://colab.research.google.com/github/OsipovOleg/crash-python-notebooks/blob/master/scientific_computing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Байесовская регрессия

Пусть дана обучающая выборка $\mathcal{D}$, состоящая из $N$ элементов, которая получена следующим образом: 

$$
y_i = w_0 + x_i w_1 + N(0, \sigma^2).
$$

Пусть априорное распределение для каждого из весов является нормальным. 

1. Получите вид апостериорного распределения $p(w| \mathcal{D})$.
2. Рассмотрите как меняется апостериорное распреление весов в зависимости *от числа объектов*, которые мы используем для обучения модели. 
- Получите параметры этого распределения. 
- Постройте плотности распределения весов. 
- Просемлируйте несколько значений весов из полученного апостериорного распределения и постройте предсказания (линии регересии на некотором интервале).
3. Как ведут себя полученные линии при добавлении новых данных. Какие выводы можно сделать. 

1. 

w_0 = -1/2 * -2 sum(y_i) (i = 0...N) /b^2 + -1/2 * 2m/d^2

w_1 = -1/2 * -2 sum(y_i) (i = 0...N) sum(x_i) (i = 0...N) /b^2 + -1/2 * 2m/d^2

w_0*w_1 = -1/2 * 2 sum(x_i) (i = 0...N)/b^2

/// подсчеты ////

мы доказали соответствие с нормальным распределением, из чего делаем вывод, что

p(w|D) = 1/(pi * q_1 * q_2 * sqrt(1 - p^2)) exp(-1/2(1 - p^2) * [(x_1 - m_1)^2/q_1^2 - p* (2 (x_1-m_1)(x_2-m_2)/(q_1 * q_2)) + (x_2-m_2)^2/q_2^2])

КОД ЧАСТЬ ПОДСЧЕТОВ РАСПРЕДЕЛЕНИЯ

In [53]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [54]:
np.random.seed(42)
true_weight = np.array([[4], [1]])
N = 10
X = np.random.normal(size = (N, 2))
X[:, 0] = 1
X[:10]

array([[ 1.        , -0.1382643 ],
       [ 1.        ,  1.52302986],
       [ 1.        , -0.23413696],
       [ 1.        ,  0.76743473],
       [ 1.        ,  0.54256004],
       [ 1.        , -0.46572975],
       [ 1.        , -1.91328024],
       [ 1.        , -0.56228753],
       [ 1.        ,  0.31424733],
       [ 1.        , -1.4123037 ]])

In [55]:
noize_sigma = 1
y = X@true_weight
y += np.random.normal(size = y.shape, scale = noize_sigma)
y[:5]

array([[5.32738447],
       [5.29725356],
       [3.83339125],
       [3.34268654],
       [3.99817732]])

In [56]:
weight_sigma = 1
weight_sigma_matrix = np.array(np.eye(2)*(weight_sigma**2))
m = 1

In [57]:
# w_0**2
xi_1 = - 1/(2*weight_sigma**2)*N - 1/(2*noize_sigma**2) 
xi_1

-5.5

In [58]:
# w_1**2
xi_2 = - 1/(2*weight_sigma**2)*X.sum() - 1/(2*noize_sigma**2)
xi_2

-4.710634737408744

In [59]:
# w_0*w_1
xi_3 = -1/2 * 2*X.sum()/(weight_sigma**2)
xi_3

-8.421269474817487

In [60]:
# w_0
xi_4 = -1/2 * -2*y.sum()/(weight_sigma**2) + -1/2 * 2*m/(noize_sigma**2)
xi_4

35.20283382812295

In [61]:
# w_1
xi_5 = -1/2 * -2*y.sum()*X.sum()/(weight_sigma**2) + -1/2 * 2*m/(noize_sigma**2)
xi_5

303.8738194186617

In [62]:
p = xi_3/(2* (xi_1 * xi_2)**(0.5))
p

-0.8272304440468725

In [63]:
b_1 = 1/((-xi_1*2*(1-p**2))**(0.5))
b_1

0.5366280719486661

In [64]:
b_2 = 1/((-xi_2*2*(1-p**2))**(0.5))
b_2

0.5798491397656008

In [65]:
a = -1 / (2 * (1-p**2)) * 1 / (weight_sigma**2)
a

-1.583833281818385

In [66]:
b = -1 / (2 * (1-p**2)) * 1 / (noize_sigma**2)
b

-1.583833281818385

In [67]:
c = 1 / (2 * (1-p**2)) * p**2 / (weight_sigma * noize_sigma)
c

1.083833281818385

In [68]:
m_1 = (xi_5 - (2*b*xi_4)/c)/(4*b/c - c)
m_1

-58.702765907483275

In [69]:
m_2 = (xi_4 + 2*a*m_1)/(-c)
m_2

-204.04763933362247